In [1]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [2]:
#Read main data
data=pd.read_excel(r'D:\Personal\University\Master\Thesis\Water Security in 1399\data_values_1399.xlsx',sheet_name='data_transpose')
rainfall_data=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\rainfall_data.xlsx',sheet_name='Sheet1')

# Read data necessary for normalization
ind_type=pd.read_excel(r'D:\Personal\University\Master\Thesis\Water Security in 1399\indicator_type_1399.xlsx',sheet_name='Sheet1')

In [3]:
# # defining normalization function for (dam capicity)/(surface rwr)
# # in this version the maximum tolerable amount of Dam capacity to 
# # srwr ratio is considered 2.3 manually to keep consistency between versions
def damcap_normalize(ser1,ser2):
    result=ser1*0
    length=ser1.size
    for i in range(length):
        if ser1[i]>=ser2[i]:
            result[i]=(2.3-ser1[i])/(2.3-ser2[i])
        else:
            result[i]=(ser1[i])/(ser2[i])
        if result[i]<0.01:
            result[i]=0.01
    return result  

#defining aggregation functions
def aggregate(df,a=0.5):
    size=len(df.columns)
    result=(df.product(axis=1)**(1/size))*a+df.mean(axis=1)*(1-a)
    return result

#defining weighted aggregation functions
def weighted_addminagg(dfi,dfw,a=0.5):
    weight_array=np.array(dfw)[0]
    dfmin=dfi.min(axis=1)
    weighted_sum=dfi.multiply(weight_array,axis='columns').sum(axis=1)
    result=a*dfmin+(1-a)*weighted_sum
    return result

def weighted_sum(dfi,dfw):
    weight_array=np.array(dfw)[0]
    weighted_sum=dfi.multiply(weight_array,axis='columns').sum(axis=1)
    return weighted_sum

def geomean(dfi):
    size=len(dfi.columns)
    geomean=(dfi.product(axis=1)**(1/size))
    return geomean

def weighted_geomean(dfi,dfw):
    weight_array=np.array(dfw)[0]
    weighted_geomean=dfi.pow(weight_array,axis='columns').product(axis='columns')
    return weighted_geomean

def weighted_addgeo(dfi,dfw,a=0.5):
    weight_array=np.array(dfw)[0]
    weighted_geomean=dfi.pow(weight_array,axis='columns').product(axis='columns')
    weighted_sum=dfi.multiply(weight_array,axis='columns').sum(axis=1)
    result= a*weighted_geomean+(1-a)*weighted_sum
    return result

In [4]:
#selecting rainfall data after 1366 (nearly 30 years)
rainfall_modified=rainfall_data.loc[(rainfall_data['syear']>1366) & (rainfall_data['syear']<1399)]

#making a dictionary of provinces and their respective data
pr=list(rainfall_modified.ostan.unique())
prdict={elem:pd.DataFrame() for elem in pr}
for key in prdict.keys():
    prdict[key]=rainfall_modified[:][rainfall_modified.ostan==key]
    
#making a dictionary consisting of province names as keys and annual precipitation as respective values
annualrain_dict={x:pd.DataFrame() for x in pr}
for key in annualrain_dict.keys():
    annualrain_dict[key]=prdict[key].groupby('syear').precnew.sum()

#calculating coefficients of variation
interannual_varicoef={x:pd.DataFrame() for x in pr}
for key in interannual_varicoef.keys():
    interannual_varicoef[key]=annualrain_dict[key].std()/annualrain_dict[key].mean()
monthly_varicoef={x:pd.DataFrame() for x in pr}
for key in monthly_varicoef.keys():
    monthly_varicoef[key]=prdict[key].precnew.std()/prdict[key].precnew.mean()
annualevap_varicoef={x:pd.DataFrame() for x in pr}

#converting monthly coefficient of variation to dataframe and sorting it based on original data
monthlyvaricoeff_df = pd.DataFrame(monthly_varicoef.items(),columns=['province', 'monthly_varicoeff'])
monthlyvaricoeff_df = monthlyvaricoeff_df.drop([31])
monthlyvaricoeff_df = monthlyvaricoeff_df.set_index('province')
monthlyvaricoeff_df = monthlyvaricoeff_df.reindex(index=data['province'])
monthlyvaricoeff_df = monthlyvaricoeff_df.reset_index()

#converting annual coefficient of variation to dataframe and sorting it based on original data
intanvaricoeff_df = pd.DataFrame(interannual_varicoef.items(),columns=['province', 'rain_coeff_variability'])
intanvaricoeff_df = intanvaricoeff_df.drop([31])
intanvaricoeff_df = intanvaricoeff_df.set_index('province')
intanvaricoeff_df = intanvaricoeff_df.reindex(index=data['province'])
intanvaricoeff_df = intanvaricoeff_df.reset_index()

# adding calculated coefficients of variation to the data
data['rain_coeff_variation']=intanvaricoeff_df['rain_coeff_variability']
data['monthly_varicoeff']= monthlyvaricoeff_df['monthly_varicoeff']

In [5]:
#Creating a dataframe consisting of necessary variables for the calculation of sub-indicators
variables=pd.DataFrame()
variables['province']=data.province
variables['irwr']=data.precipitation-data.evaporation
variables['surf_irwr']=(data.precipitation-data.evaporation)*data.runoff_coeff
variables['withdraw_surf']=data.iloc[:,5:8].sum(axis=1)
variables['gw_irwr']=(data.precipitation-data.evaporation)*(1-data.runoff_coeff)
variables['withdraw_gw']=data.iloc[:,8:11].sum(axis=1)
variables['withdraw_agr']=data.withdraw_gw_agr+data.withdraw_surf_agr
variables['withdraw_ind']=data.withdraw_gw_ind+data.withdraw_surf_ind
variables['access_sanitation_total']=(data.access_sanitation_urban*data.urban_pop_ratio+data.access_sanitation_rural*(1-data.urban_pop_ratio))
variables['deficit_gw_annual']=data.annual_gw_variation*(-1)
variables['deficit_gw_aggregate']=data.aggregate_gw_variation*(-1)
variables['withdraw_total']=np.NaN
variables['withdraw_total']=data[['withdraw_surf_agr','withdraw_surf_ind','withdraw_surf_dom','withdraw_gw_agr','withdraw_gw_ind','withdraw_gw_dom']].sum(axis=1)
variables['agr_withdraw_ratio']=variables['withdraw_agr']/variables['withdraw_total']
variables['access_sanitation_total']=data['access_sanitation_urban']*data['urban_pop_ratio']+data['access_sanitation_rural']*(1-data['urban_pop_ratio'])

#setting negative deficit values equal to zero
variables.loc[variables['deficit_gw_annual'] < 0 ,'deficit_gw_annual']=0
variables.loc[variables['deficit_gw_aggregate'] < 0 ,'deficit_gw_aggregate']=0
variables['withdraw_gw_allowable']=variables.gw_irwr-(variables.deficit_gw_aggregate/17)
variables['withdraw_agr_decrease']=variables.withdraw_gw-variables.withdraw_gw_allowable
variables.loc[variables['withdraw_agr_decrease']<0,'withdraw_agr_decrease']=0
variables['agrwat_lost_ratio']=variables.withdraw_agr_decrease/variables.withdraw_agr


In [6]:
#Creating a dataframe for the sub-indicators
#Dimension 1: Resources
sub_indicators=pd.DataFrame()
sub_indicators['rain_coeff_variation']=data['rain_coeff_variation']
sub_indicators['monthly_varicoeff']=data['monthly_varicoeff']
sub_indicators['gw_agdef_gwrwr']=variables.deficit_gw_aggregate/variables.gw_irwr
sub_indicators['gw_andef_withdraw']=variables.deficit_gw_annual/variables.withdraw_gw
sub_indicators['agr_dependency_gw']=data.withdraw_gw_agr/variables.withdraw_agr
sub_indicators['ind_dependency_gw']=data.withdraw_gw_ind/variables.withdraw_ind
sub_indicators['urbanwat_dependency_gw']=data.withdraw_urban_gw/data.produced_urban_wat
sub_indicators['ruralwat_dependency_gw']=data.withdraw_rural_gw/data.produced_rural_wat
sub_indicators['withdraw_surf_ratio']=variables.withdraw_surf/variables.surf_irwr
sub_indicators['withdraw_gw_ratio']=variables.withdraw_gw/variables.gw_irwr
sub_indicators['anomaly_rain']=abs(data['anomaly_rain'])
sub_indicators['anomaly_temp']=data['anomaly_temp']
sub_indicators['irwr_percap']=(variables.irwr*(10**6)/data.population)/(1+data['pop_growth']/100)
#Dimension 2: access

sub_indicators['access_wat_urban']=data['access_wat_urban']
sub_indicators['access_wat_rural']=data['access_wat_rural']
sub_indicators['under_stress_pop']=data['under_stress_pop']
sub_indicators['access_sanitation_urban']=data['access_sanitation_urban']
sub_indicators['access_sanitation_rural']=data['access_sanitation_rural']
sub_indicators['treated_municipal_wastewater']=data.waste_facility_cap/data.daily_produced_waste
sub_indicators['quality_proxy']=data.urban_fam_treatwat/data.total_urban_fam
sub_indicators['damcap_rwr_ratio']=data.dam_cap/variables.surf_irwr

#Dimension 3:Economy
sub_indicators['efficiency_agr']=data.agr_added_value/variables.withdraw_agr
sub_indicators['modern_irrig']=data.land_irrig_modern/data.land_irrig_tot
sub_indicators['employment_lost_agr']=variables.agrwat_lost_ratio*data.agr_employment
sub_indicators['unaccounted_wat_urban']=data['unaccounted_wat_urban']
sub_indicators['unaccounted_wat_rural']=data['unaccounted_wat_rural']
sub_indicators['efficiency_ind']=data.ind_added_value/variables.withdraw_ind

sub_indicators.index=variables['province']
sub_indicators.loc['khuz','damcap_rwr_ratio']=1

In [7]:
#Normalizing Data
normalized=sub_indicators*0
max_allowable_withdraw=variables['withdraw_gw_allowable']/variables['gw_irwr']
max_allowable_withdraw.index=sub_indicators.index

for col in ind_type.columns:
    i=ind_type.columns.get_loc(col)
    if ind_type.loc[2,col]=='b': # The bigger the better indicators
        normalized.loc[sub_indicators[col]>=ind_type.loc[0,col],col]=1
        normalized.loc[sub_indicators[col]<=ind_type.loc[1,col],col]=0.01
        cond=(sub_indicators[col]>ind_type.loc[1,col])& (sub_indicators[col]<ind_type.loc[0,col])
        normalized.loc[cond,col]=(sub_indicators.loc[cond,col]-ind_type.loc[1,col])/(ind_type.loc[0,col]-ind_type.loc[1,col])
    elif ind_type.loc[2,col]=='l':      # The lower the better indicators
        normalized.loc[sub_indicators[col]<=ind_type.loc[0,col],col]=1
        normalized.loc[sub_indicators[col]>=ind_type.loc[1,col],col]=0.01
        cond=(sub_indicators[col]<ind_type.loc[1,col])& (sub_indicators[col]>ind_type.loc[0,col])
        normalized.loc[cond,col]=(ind_type.loc[1,col]-sub_indicators.loc[cond,col])/(ind_type.loc[1,col]-ind_type.loc[0,col])
    elif ind_type.loc[2,col]=='diff':    # GW withdrawal to rwr ratio normalization
        cond1=(sub_indicators[col] > max_allowable_withdraw)
        normalized.loc[cond1,col]=0.01
        cond2=sub_indicators[col]<0.25
        normalized.loc[cond2,col]=1
        cond3=(sub_indicators[col] < max_allowable_withdraw) & (sub_indicators[col]>0.25)
        normalized.loc[cond3,col]=(max_allowable_withdraw.loc[cond3]-sub_indicators.loc[cond3,col])/(max_allowable_withdraw.loc[cond3]-0.25)    

In [8]:

variation_coeff_agg=pd.Series(sub_indicators[['rain_coeff_variation','monthly_varicoeff']].mean(axis=1),index=variables['province'])
variation_coeff_bins=pd.cut(variation_coeff_agg,4,labels=['Q1','Q2','Q3','Q4'])


In [9]:
# dam capacity to surface rwr ratio normalization    
variation_coeff_agg=pd.Series(sub_indicators[['rain_coeff_variation','monthly_varicoeff']].mean(axis=1),index=variables['province'])
variation_coeff_bins=pd.cut(variation_coeff_agg,4,labels=['Q1','Q2','Q3','Q4'])
ideal_damcap=pd.Series(index=variables['province'],dtype='float64') 
ideal_damcap[variation_coeff_bins=='Q1']=1
ideal_damcap[variation_coeff_bins=='Q2']=0.9
ideal_damcap[variation_coeff_bins=='Q3']=0.8
ideal_damcap[variation_coeff_bins=='Q4']=0.7
normalized['damcap_rwr_ratio']=damcap_normalize(sub_indicators['damcap_rwr_ratio'],ideal_damcap)   
normalized.loc['khuz','damcap_rwr_ratio']=1

# modifying modern irrigation indicator vlues based on aggregate GW reservoir deficits
gw_deficit_agg=pd.Series(aggregate(normalized[['gw_agdef_gwrwr','gw_andef_withdraw']]),index=variables['province'])
gw_deficit_bins=pd.cut(gw_deficit_agg,4,labels=['Q1','Q2','Q3','Q4'])
gw_modifier=pd.Series(index=variables['province'],dtype='float64')
gw_modifier[gw_deficit_bins=='Q1']=0.7
gw_modifier[gw_deficit_bins=='Q2']=0.8
gw_modifier[gw_deficit_bins=='Q3']=0.9
gw_modifier[gw_deficit_bins=='Q4']=1
normalized['modern_irrig']=normalized['modern_irrig'].multiply(gw_modifier)

for col in normalized.columns:
    normalized.loc[normalized[col]<0.01,col]=0.01


In [10]:
res_sub_columns=['ACV','MCV','Dag','Dan','GWDagr','GWDind','GWDurb','GWDrur','SWS','GWS','APA','ATA','IRWR']
acc_sub_columns=['WAurb','WArur','USP','SAurb','SArur','TPC','WQ','DC']
eco_sub_columns=['AE','MI','EL','NRWurb','NRWrur','IE']
sub_columns1_list= ['R1','R1','R2','R2','R3','R3','R3','R3','R4','R4','R5','R5','R7']+['A1','A1','A1','A2','A2','A2','A3','A3']+['E1','E1','E1','E2','E2','E3']
sub_columns2_list=res_sub_columns+acc_sub_columns+eco_sub_columns
arraycolumns=[sub_columns1_list]+[sub_columns2_list]
normalized_pp=pd.DataFrame(normalized).round(decimals=4)
normalized_pp.columns=sub_columns2_list
# normalized_pp.loc[:,[('E1','AE'),('E1','MI'),('E1','EL'),('E2','NRWurb'),('E2','NRWrur'),('E3','IE')]]


In [11]:
# normalized_lvl2=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
# normalized_lvl2.columns=pd.MultiIndex.from_arrays([['R5'],['CV']], names=('Aggregation Level', 'Indicator'))
# normalized_lvl2[('R5','CV')]=geomean(normalized_ahp.loc[:,'R1'],ahp_weights['R1']).round(decimals=4)
# normalized_lvl2[('R6','GD')]=weighted_sum(normalized_ahp.loc[:,'R2'],ahp_weights['R2']).round(decimals=4)
# normalized_lvl2[('R6','GWD')]=weighted_sum(normalized_ahp.loc[:,'R3'],ahp_weights['R3']).round(decimals=4)
# normalized_lvl2.head()

In [21]:
## Assessing water security using ageomtric mean aggregation rules

#  calculating indicators 
indicators_pp=pd.DataFrame(index=sub_indicators.index)
temp=pd.DataFrame(index=sub_indicators.index)
temp['GD']=geomean(normalized_pp[['Dag','Dan']])
temp['GWD']=geomean(normalized_pp[['GWDagr','GWDind','GWDurb','GWDrur']])
# indicators_pp.columns=pd.MultiIndex.from_arrays([['Resource'],['IRWR']], names=('Dimension', 'Indicator'))
indicators_pp['IRWR']=normalized_pp['IRWR']
indicators_pp['CC']=geomean(normalized_pp[['ACV','MCV','APA','ATA']]).round(decimals=4)
indicators_pp['WS']=geomean(normalized_pp[['SWS','GWS']]).round(decimals=4)
# if a province has low groundwater reservoir deficit then high dependency on groundwater is considered okay i.e. if GD> 0.8 then GWD not involved in aggregation
indicators_pp.loc[temp['GD']<0.8,'GW']=geomean(temp[['GD','GWD']]).round(decimals=4)
indicators_pp.loc[temp['GD']>0.8,'GW']=temp['GD']
indicators_pp['DW']=geomean(normalized_pp[['WAurb','WArur','USP','WQ']]).round(decimals=4)
# indicators_pp['WQ']=normalized_pp['WQ']
indicators_pp['DC']=normalized_pp['DC']
indicators_pp['SS']=geomean(normalized_pp[['SAurb','SArur','TPC']]).round(decimals=4)
indicators_pp['Eagr']=geomean(normalized_pp[['AE','MI','EL']]).round(decimals=4)
indicators_pp['IE']=normalized_pp['IE']
indicators_pp['NRW']=geomean(normalized_pp[['NRWurb','NRWrur']]).round(decimals=4)
indicators_pp

,IRWR,CC,WS,GW,DW,DC,SS,Eagr,IE,NRW
province,,,,,,,,,,
azsh,0.2670,0.5928,0.1935,0.459800,0.6062,0.4214,0.2046,0.5694,0.1689,0.4793
azgh,0.6868,0.4428,0.0792,0.833340,0.7609,0.5425,0.2625,0.3323,0.0639,0.5900
arda,0.3400,0.6082,0.0745,0.522400,0.5632,0.4082,0.1849,0.3275,0.0199,0.4349
esfe,0.3901,0.4389,0.0926,0.126200,0.8014,0.6060,0.2500,0.3524,1.0000,0.6849
albo,0.0100,0.5203,0.0878,0.084300,0.4816,0.8888,0.1673,0.4926,0.0773,0.3046
ilam,1.0000,0.2364,0.7623,0.946044,0.8377,0.2439,0.1322,0.4782,0.0100,0.5266
bush,0.2345,0.0750,0.0100,0.531400,0.2224,0.4716,0.0638,0.5324,0.1793,0.2068
tehr,0.0100,0.5620,0.0100,0.098900,0.7797,0.4441,0.3423,0.4773,0.3582,0.5403
chba,1.0000,0.3976,1.0000,0.829659,0.7622,0.4581,0.1338,0.6161,0.0671,0.4346


In [13]:
# #  calculating dimensions
Dimensions=pd.DataFrame(index=sub_indicators.index)
Dimensions['Resource']=geomean(indicators_pp[['IRWR','CC','WS','GW']]).round(decimals=4)
Dimensions['Access']=geomean(indicators_pp[['DW','DC','SS']]).round(decimals=4)
Dimensions['Economy']=geomean(indicators_pp[['Eagr','IE','NRW']]).round(decimals=4)

# #  calculating WSI
Dimensions['WSI']=geomean(Dimensions[['Resource','Access','Economy']]).round(decimals=4)

Dimensions

,Resource,Access,Economy,WSI
province,,,,
azsh,0.3445,0.3739,0.3586,0.3588
azgh,0.3764,0.4767,0.2323,0.3467
arda,0.2995,0.3490,0.1415,0.2455
esfe,0.2115,0.4952,0.6226,0.4025
albo,0.0788,0.4153,0.2264,0.1949
ilam,0.6426,0.3000,0.1360,0.2971
bush,0.0983,0.1884,0.2703,0.1711
tehr,0.0486,0.4912,0.4520,0.2210
chba,0.7579,0.3602,0.2619,0.4150


In [14]:
# Dimensions.to_excel('geo_conf_1399.xlsx')

In [17]:
Res_vs_subs=normalized_pp.iloc[:,:13].join(Dimensions['Resource'])
Acc_vs_subs=normalized_pp.iloc[:,13:21].join(Dimensions['Access'])
Eco_vs_subs=normalized_pp.iloc[:,21:].join(Dimensions['Economy'])
Res_corr=Res_vs_subs.corr(method='pearson')['Resource']
Acc_corr=Acc_vs_subs.corr(method='pearson')['Access']
Eco_corr=Eco_vs_subs.corr(method='pearson')['Economy']

In [24]:
Res_vs_subs=indicators_pp.iloc[:,:4].join(Dimensions['Resource'])
Res_corr=Res_vs_subs.corr(method='pearson')['Resource']
Acc_vs_subs=indicators_pp.iloc[:,4:7].join(Dimensions['Access'])
Acc_corr=Acc_vs_subs.corr(method='pearson')['Access']
Eco_vs_subs=indicators_pp.iloc[:,7:].join(Dimensions['Economy'])
Eco_corr=Eco_vs_subs.corr(method='pearson')['Economy']
Dimensions.corr(method='pearson')['WSI']

Resource    0.661064
Access      0.517773
Economy     0.483038
WSI         1.000000
Name: WSI, dtype: float64